## I need to answer several questions. The answers to those questions must be supported by data and analytics. These are the questions:

### 1. Which type of complaint should the Department of Housing Preservation and Development of New York City focus on first?
### 2. Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1?
### 3. Does the Complaint Type that you identified in response to question 1 have an obvious relationship with any particular characteristic or characteristics of the houses or buildings?
### 4. Can a predictive model be built for a future prediction of the possibility of complaints of the type that you have identified in response to question 1?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df= pd.read_csv('/kaggle/input/nyc-311-hpd-calls/311_Service_Requests_from_2010_to_Present.csv')
print(df.shape)
df.head()

In [ ]:
df.columns

### Let's get rid of all the unnecessary fields

In [ ]:
df=df[['Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name','Address Type',
       'City', 'Status', 'Due Date',
       'Resolution Description','Borough',
       'Latitude', 'Longitude']]
df.head()

In [ ]:
df[['Address Type']].describe()

### The field "Address Type" seems to have only one value. It's not useful information. We will LET IT GO

In [ ]:
df=df.drop(columns=['Address Type'])

In [ ]:
df_comp=df.groupby('Complaint Type')[["Unique Key"]].count()
df_comp.sort_values('Unique Key',ascending=False).head()

In [ ]:
df_comp=df_comp[df_comp['Unique Key']>80000].sort_values(by='Unique Key',ascending=False)
df_comp.columns=['No of complaints']
df_comp.T

# ***It seems like the highest complaints are for HEAT or HOT WATER!!***

### We are only going to focus on the most frequent occuring problems

In [ ]:
print(df.shape)
df=df[df['Complaint Type'].isin(df_comp.index)]
print(df.shape)
df.head()

### Which area had the largest number of complaints?

In [ ]:
df_bor= df.groupby('Borough')[['Unique Key']].count().sort_values('Unique Key',ascending=False)
df_bor

## It seems like Brooklyn has the highest number of complaints. But BRONX is also very close. There are also some unspecified entries. We will have to find what borough those zip numbers belong to.

In [ ]:
df_pluto = pd.read_csv('/kaggle/input/nyc-pluto/pluto_20v2.csv')
df_pluto.head()

In [ ]:
df_zip=df.groupby('Incident Zip')[['Borough']].agg(lambda x:x.value_counts().index[0])
df_zip.head()

In [ ]:
for i,j in zip(df[df['Borough']=='Unspecified'].index,df[df['Borough']=='Unspecified']['Incident Zip']):
    if np.isnan(j):
        continue
    df.at[i,'Borough']=df_zip.at[j,'Borough']
    #print(type(j))
    


In [ ]:
df[df['Borough']=='Unspecified'].head()

In [ ]:
df.groupby('Borough')[['Unique Key']].count().sort_values('Unique Key',ascending=False)

## We still have some Unspecified values. But they are because their zip wasn't given. Since their number is now less by multiple factors of ten, we will ignore the rest.

### Let's see which address has the most complaints

In [ ]:
df.groupby('Incident Address')[['Unique Key']].count().sort_values('Unique Key',ascending=False)

In [ ]:
df[df['Incident Address']=='34 ARDEN STREET'][['Incident Address','Incident Zip','Borough']].head(1)


### The address where most number of complaints came from is 
# 34 ARDEN STREET, MANHATTAN 10040

In [ ]:
df.groupby('Incident Zip')[['Unique Key']].count().sort_values('Unique Key',ascending=False).head()

## The zipcode where the most number of complaints came from is 
# 11226

In [ ]:
df.groupby('Status')[['Unique Key']].count().sort_values('Unique Key',ascending=False).head()

In [ ]:
df_pluto=df_pluto[['address','bldgarea','bldgdepth','builtfar','commfar','facilfar','lot','lotarea',
                   'lotdepth','numbldgs','numfloors','officearea','resarea','residfar','retailarea',
                   'yearbuilt','yearalter1','zipcode','ycoord','xcoord']]
df_pluto.shape

In [ ]:
df_pluto['bldgage']=2020-df_pluto['yearbuilt']
df_pluto.head()

In [ ]:
df_comp_count=df.groupby('Incident Address')[['Incident Address']].count()

In [ ]:
df_comp_count.columns=['count of complaints']
df_comp_count['address']=df_comp_count.index
df_comp_count.head()

In [ ]:
#df_comp_count.index=None
df_comp_count.reset_index(drop=True,inplace=True)
df_comp_count.head()

In [ ]:
df_corr = pd.merge(df_comp_count,df_pluto,on='address')
df_corr.head()

In [ ]:
print(df_comp_count.shape)
print(df_pluto.shape)
df_corr.shape

In [ ]:
df_corr.corr()